# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [6]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import ollama

In [9]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Make a long joke"}
]

#payload = {
#        "model": MODEL,
#        "messages": messages,
#        "stream": True
#    }

def call_ollama(messages):
    response = ollama.chat(
        messages=messages,
        model="llama3.2",
        stream=True
    )

    for chunk in response:
        content = chunk['message']['content']
        print(content, end='', flush=True) 
call_ollama(messages)

Here's one:

A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?"

The librarian replied, "It rings a bell, but I'm not sure if it's here or not."

The man laughed and said, "Well, that's a pretty paws-itive review!"

But just as he was about to leave, the librarian's manager came over and whispered something in her ear. The manager then looked at the man and said, "I'm afraid we've been cat-ched off guard by your joke, but I think we can take it one step further."

The librarian smiled mischievously and said, "Why don't you paws for a moment while we check if the book is indeed here?"

So the man waited patiently while the librarian checked the shelves. After what felt like an eternity, she finally came back with a book on Pavlov's dogs and Schrödinger's cat.

The man exclaimed, "Wow, I guess my joke really did ring true!"

But as he was about to open the book, the librarian's manager appeared once again and whispered som

In [2]:
# Initialize and constants

openai = OpenAI()

API key looks good so far


In [10]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [11]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [22]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You must respond only in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [13]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [14]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [15]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [25]:
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ]
    )
    result = response['message']['content']
    return json.loads(result)

In [17]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/moonshotai/Kimi-K2-Instruct',
 '/mistralai/Voxtral-Mini-3B-2507',
 '/mistralai/Voxtral-Small-24B-2507',
 '/moonshotai/Kimi-K2-Base',
 '/HuggingFaceTB/SmolLM3-3B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/FunAudioLLM/ThinkSound',
 '/spaces/llamameta/Grok-4-heavy-free',
 '/spaces',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/common-pile/caselaw_access_project',
 '/datasets/microsoft/rStar-Coder',
 '/datasets/HuggingFaceTB/smoltalk2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly

In [18]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about me page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'careers/jobs page',
   'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'blog/posts page', 'url': 'https://edwarddonner.com/posts/'}]}

In [23]:
get_links("https://huggingface.co")

{
    "links": [
        {"type": "About page", "url": "https://huggingface.co"},
        {"type": "Company page", "url": "https://huggingface.co/brand"},
        {"type": "Blog", "url": "https://blog.huggingface.co"},
        {"type": "GitHub", "url": "https://github.com/huggingface"},
        {"type": "Twitter", "url": "https://twitter.com/huggingface"},
        {"type": "LinkedIn", "url": "https://www.linkedin.com/company/huggingface/"},
        {"type": "Discord", "url": "https://huggingface.co/join/discord"},
        {"type": "Zhihu", "url": "https://www.zhihu.com/org/huggingface"},
        {"type": "Documentation", "url": "https://docs.huggingface.co/"},
        {"type": "Learn more", "url": "https://huggingface.co/learn"}
    ]
}


{'links': [{'type': 'About page', 'url': 'https://huggingface.co'},
  {'type': 'Company page', 'url': 'https://huggingface.co/brand'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'Zhihu', 'url': 'https://www.zhihu.com/org/huggingface'},
  {'type': 'Documentation', 'url': 'https://docs.huggingface.co/'},
  {'type': 'Learn more', 'url': 'https://huggingface.co/learn'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [26]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [27]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/about'}, {'type': 'Company page', 'url': 'https://huggingface.co/company'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog page', 'url': 'https://discuss.huggingface.co'}, {'type': 'News/Status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
moon

In [28]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog page', 'url': 'https://blog.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Discord server', 'url': 'https://huggingface.co/join/discord'}, {'type': 'Zhihu organization page', 'url': 'https://www.zhihu.com/org/huggingface'}]}


ConnectionError: HTTPSConnectionPool(host='blog.huggingface.co', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001B3F905F410>: Failed to resolve 'blog.huggingface.co' ([Errno 11001] getaddrinfo failed)"))

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [23]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
### The AI Community Building the Future

At Hugging Face, we are more than just a cutting-edge platform for machine learning (ML) and artificial intelligence (AI). We are a vibrant community of innovators and collaborators dedicated to transforming the future of technology through open-source practices. Our mission is to enable everyone—from developers to enterprises—to create, discover, and share powerful machine-learning models, datasets, and applications.

![Hugging Face](https://huggingface.co/front/assets/huggingface_logo.svg) 

---

## What We Offer

### Extensive Resources
- **1M+ Models**: Explore a vast collection of models for NLP and beyond.
- **250k+ Datasets**: Access diverse datasets to fuel your ML projects.
- **400k+ Applications**: Browse and deploy innovative AI applications through our platform.

### Collaborative Environment
Our platform hosts an array of tools designed for collaboration and innovation. Engage with the community to build, share, and improve ML models and applications.

### Enterprise Solutions
For organizations, we offer paid compute and enterprise solutions, ensuring you have access to robust, scalable AI services with dedicated support and advanced security. 

---

## Company Culture

At Hugging Face, we believe in fostering a culture of openness, collaboration, and continuous learning. We prioritize inclusivity and encourage team members to share ideas, which drives our innovation. Our existing team enjoys flexible working environments, emphasizing creativity and independent exploration within the AI domain.

### Join Our Community
We are dedicated to building a diverse workforce that amplifies unique perspectives in AI. Explore our [careers page](#) for the latest job openings and become part of our mission to democratize AI.

---

## Who Trusts Us
More than **50,000 organizations** utilize Hugging Face, including:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Grammarly**

Our commitment to open-source ML solutions makes us a trusted partner for enterprises across various sectors.

---

## Get Involved
Whether you are looking to advance your career, utilize our advanced AI solutions, or simply explore the rich landscape of ML applications, Hugging Face is the perfect place for you. 

- **Sign Up** to explore our offerings and start contributing today!
- For enterprise needs, [contact us](#) for personalized solutions.

Together, let's build the future of artificial intelligence!

--- 

*For more information, visit [huggingface.co](https://huggingface.co).*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face!

![Hugging Face Logo](https://huggingface.co/logo.png)

### The AI Community Building the Future

At Hugging Face, we are at the forefront of the AI revolution, dedicated to fostering a collaborative ecosystem for machine learning (ML) practitioners and enthusiasts alike. Our platform uniquely caters to the needs of developers, researchers, and enterprises by building tools that allow everyone to create, discover, and collaborate on state-of-the-art ML models and datasets.

---

## What We Offer

### **Models**
With over **1 million models** at your fingertips, you can browse through cutting-edge AI solutions such as ChatGPT and image generators. Stay updated with **trending models** and see what the community is developing in real-time.

### **Datasets**
Access and share **250,000+ datasets** for a wide array of ML tasks. We provide a robust framework for testing and improving models, ensuring a high-quality resource for developers.

### **Spaces**
Deploy and share applications fast! Our **Spaces** feature allows developers to create and run interactive ML applications effortlessly.

---

## Our Customers

More than **50,000 organizations** use Hugging Face for their AI needs, including some of the world's most innovative companies, such as:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**

Each organization leverages our powerful tools to build scalable AI solutions while benefiting from our community-driven approach.

---

## Company Culture

At Hugging Face, we pride ourselves on a community-focused culture. We believe in the power of collective intelligence; thus, collaboration and open-source contributions are at our core. We encourage innovation and the pursuit of knowledge, leading to an inclusive environment where diverse ideas can flourish.

---

## Join Our Team

Hugging Face is on a mission to make AI accessible and useful to everyone. We are continuously looking for passionate individuals who want to be a part of this journey. 

### **Current Openings:**
- Machine Learning Engineers
- Data Scientists
- Software Developers
- Community Managers

**Why Work With Us?**
- Collaborate with industry leaders in AI.
- Work remotely and enjoy flexible hours.
- Be part of a culture that promotes learning and growth.

**Explore Careers:** [View Current Openings](https://huggingface.co/careers)

---

## Get In Touch

Whether you're an investor, a developer, or a job seeker, we invite you to connect with us. Explore our offerings and become a part of the Hugging Face community!

**Follow Us:**  
- [GitHub](https://github.com/huggingface)  
- [Twitter](https://twitter.com/huggingface)  
- [LinkedIn](https://www.linkedin.com/company/huggingface)  
- [Discord](https://discord.com/invite/huggingface)

**Website:** [Hugging Face](https://huggingface.co)

---

*Together, let’s build the future of AI!*



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>